In [17]:
import webbrowser
import folium, json
import pandas as pd


m = folium.Map(location=[35.1753194,129.0613623], #지도 중앙값 좌표
               tiles="OpenStreetMap", #띄울 지도 종류 : 한글 호환 지도
               zoom_start=11) #지도 줌 정도
geo_data = json.load(open('busan_gu.json', encoding='utf-8')) # json파일 불러오기

#--------------------------------DB연동--------------------------------
import cx_Oracle
 
# conn = cx_Oracle.connect('접속이름/비밀번호@호스트이름:포트번호/SID)순으로 작성
conn = cx_Oracle.connect('min/1234@192.168.0.6:1521/xe')
cursor = conn.cursor() # 커서 생성

#select로 값 불러오기
sql =   """
        SELECT A.SGG_CD, A.SGG_NM, D.VSTR_TOTAL
        FROM DM_VS_SGG_VSTR_SL D,
            (SELECT SGG_CD, SGG_NM
            FROM AM_CODE
            GROUP BY SGG_CD, SGG_NM) A
        WHERE D.SGG_CD = A.SGG_CD
        """
# print(sql)

# data=(0.1072,0.0862,0.0746,0.1461,0.0872,0.2942,0.2045)

cursor.execute(sql) #쿼리 실행

df = [] #데이터를 저장할 리스트 생성
for row in cursor:
#         print(row)
#         df = list((row))
        df.append(row) #df에 row값 저장
# print(df)
cursor.close()
conn.close()

column = ['sgg_cd','sgg_nm','vstr_expn'] #컬럼명
busan = pd.DataFrame(df,columns=column) #데이터 프레임으로 생성
# print(busan)

#Choropleth 레이어를 만들고, 맵 m에 추가
folium.Choropleth(geo_data=geo_data, #지도 데이터 파일 경로 
                  data = busan, #시각화 하고자 하는 데이터파일. (pandas.DataFrame)
                  columns=['sgg_nm','vstr_expn'], #[지도 데이터와 매핑할 값, 시각화 하고자하는 변수]
                  key_on='feature.id', #[feature.데이터 파일과 매핑할 값]
                  fill_color='PuBuGn', #시각화에 쓰일 색상
                  fill_opacity=0.7, #투명도
                  legend_name='부산 방문자 수(만명)' #컬러 범주 이름
                  ).add_to(m)

# m.save('folium_kr.html') #html로 저장
# webbrowser.open_new("folium_kr.html") #webbrowser로 오픈
m